防災アプリAdventCalender2024 24日目の記事で作成した深発地震の予測モデル

### 事前準備

1. リポジトリから学習用パッケージをインストール
2. 簡易版学習用データ、予測地点データを用意

In [ ]:
%pip install git+https://github.com/kotoho7/asid-predict

In [ ]:
!mkdir -p ./data ./out
!wget https://gist.githubusercontent.com/kotoho7/e1388671e5f990585e921f640b708872/raw/train_data_sample.json -O ./data/train_data.json
!wget https://gist.githubusercontent.com/kotoho7/e1388671e5f990585e921f640b708872/raw/kik.json -O ./data/predict.json

### モデルの学習

In [ ]:
from asid_predict import execute_training_process

# モデルの学習
model = execute_training_process(
    train_json_path="./data/train_data.json", # 学習用データのパス
    save_path="./out", # モデルの保存先
    target_is_pasific_plate=True, # 太平洋プレートの地震を学習
    epochs=20, # エポック数
    batch_size=64, # バッチサイズ
)

### 学習済みモデルを使って予想震度を計算

In [ ]:
import json

from asid_predict.prediction import predict_intensities
from asid_predict.dataclass import Earthquake, ObservationPoint

# 予測地点リスト
with open("./data/predict.json") as f:
    dict = json.load(f)
    stations = [ObservationPoint(**d) for d in dict]

# 震源情報
earthquake = Earthquake(
    lat=33.840,
    lon=137.423,
    depth=362,
    magnitude=6.4,
)

# 予測を実行
intensities = predict_intensities(
    model=model,
    targets=stations,
    eq=earthquake,
)

### 予測結果の表示

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

x, y, c = np.array([s.lon for s in stations]), np.array([s.lat for s in stations]), np.array(intensities)
sorted_indices = np.argsort(c)

# 散布図を作成
plt.scatter(x[sorted_indices], y[sorted_indices], c=c[sorted_indices], cmap="jet", vmin=-3, vmax=7)
plt.colorbar(label="Seismic Intensity")
plt.scatter(earthquake.lon, earthquake.lat, color="red", marker="*", s=200, label="Epicenter")
plt.title(f"Earthquake at ({earthquake.lat}, {earthquake.lon}), Depth: {earthquake.depth} km, Magnitude: {earthquake.magnitude}")
plt.xlim(128, 146)
plt.ylim(30, 46)
plt.legend()
plt.show()